Check which version is installed on your machine and please upgrade if needed. 

In [ ]:
import plotly
plotly.__version__

Now let's load the dependencies/packages that we need in order to get a simple stream going.

In [2]:
import plotly.plotly as py  
import plotly.tools as tls   
import plotly.graph_objs as go
import numpy as np 

#### Overview

In this example we're going to randomly draw two airports from a dataframe, then have our plane fly from one to the other. Then as soon as it reaches its destination, we will randomly select a new airport for our plane to fly to. 
The dataset we will be using will be based on American airports.

#### Getting Set Up

Let's load the dataset that we'll be using:

In [3]:
import pandas as pd
dframe = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/2011_february_us_airport_traffic.csv')

Let's get at least two streaming tokens for this task.

In [4]:
stream_tokens = tls.get_credentials_file()['stream_ids']
token = stream_tokens[-2]   # I'm getting my stream tokens from the end to ensure I'm not reusing tokens
print token


4lm5a0gsr8


Now let's create some `stream id objects` for each token.

In [5]:
stream_id = dict(token=token, maxpoints=3)
stream_id

{'maxpoints': 3, 'token': u'4lm5a0gsr8'}

To set this plot up, we will first create a scattergeo trace in our data list. Next we will adjust the layout in order to change the plotting surface to a map of the United States, along with some other aesthetic options. 

In [6]:
data = [dict(
        type='scattergeo',
        lon=[],
        lat=[],
        mode='markers',
        marker=dict(
            size=8,
            opacity=0.8,
            reversescale=True,
            autocolorscale=False,
            line=dict(
                width=1,
                color='rgba(102, 102, 102)'
            ),
        ),
        stream=stream_id,
        name="Plane")]

layout = dict(
        title = 'Busy Airplane Streaming',
        colorbar = False,
        geo = dict(
            scope='usa',
            projection=dict( type='albers usa' ),
            showland = True,
            landcolor = "rgb(250, 250, 250)",
            subunitcolor = "rgb(217, 217, 217)",
            countrycolor = "rgb(217, 217, 217)",
            countrywidth = 0.5,
            subunitwidth = 0.5
        ),
    )

fig = dict( data=data, layout=layout )
py.iplot( fig, validate=False, filename='geo-streaming2', auto_open=False, fileopt='extend')


Now let's set up a `stream link object` for our geo-scatter trace and start streaming some data to our plot

In [7]:
s = py.Stream(stream_id=token)

#### Start Streaming

Below we'll wrap the relevant code to generate the data stream in a function we call `lets_stream()`.
We will also follow up with a while loop that will keep our stream up persistently. Notice that the small green dot is the departing airport, the red dot is the arrival airport, and the blue dot is the plane!

In [ ]:
import datetime
import time

def lets_stream():

    s.open()

    airports = dframe.sample(4)[['lat', 'long', 'airport']]
    depart = airports.iloc[0]
    arrive = airports.iloc[1]
    num_steps = 20

    while True:

        count = 0

        lats = np.linspace(depart['lat'], arrive['lat'], num_steps)
        lons = np.linspace(depart['long'], arrive['long'], num_steps)

        for i, j in zip(lats, lons):

            # added pts for the departure and arrival airports!!!
            s.write(dict(lon=[depart['long'], j, arrive['long']],
                         lat=[
                             depart['lat'], i, arrive['lat']], type='scattergeo',
                         marker={'size': [5, 7 + 0.2 * count, 5], 'sizemode': 'area',
                                 'color': ["green", "blue", "red"],
                                 'symbol': ["circle", "star", "x-open"]},
                         text=[depart['airport'],
                               '{},{}'.format(count, datetime.datetime.now()),
                               arrive['airport']]))

            count += 1
            stall = np.random.normal(10, 3)
            time.sleep(int((abs(stall) + 0.01) / 2.0))
            s.heartbeat()
            time.sleep(int((abs(stall) + 0.01) / 2.0))

        depart = arrive
        arrive = dframe.sample(1)[['lat', 'long', 'airport']].iloc[0]

while True:
    try:
        lets_stream()
    except Exception as e:
        with open('log.txt', 'a+') as f:
            f.write(str(e))
        print(str(e))
        s.close()

You can see a live demo of the stream below:

In [4]:
tls.embed('streaming-demos','121')

In [1]:
from IPython.display import display, HTML

display(HTML('<link href="//fonts.googleapis.com/css?family=Open+Sans:600,400,300,200|Inconsolata|Ubuntu+Mono:400,700" rel="stylesheet" type="text/css" />'))
display(HTML('<link rel="stylesheet" type="text/css" href="http://help.plot.ly/documentation/all_static/css/ipython-notebook-custom.css">'))

! pip install publisher --upgrade
import publisher
publisher.publish(
    'geo-streaming', 'python/geo-streaming//', 'Streaming to a Map',
    'Streaming in Plotly with Python', name="Streaming to Maps",
    title = 'Geo-Streaming with Plotly',
    thumbnail='', language='python',
    layout='user-guide', has_thumbnail='false',
    ipynb= '~notebook_demo/82') 

Requirement already up-to-date: publisher in /Users/brandendunbar/Desktop/test/venv/lib/python2.7/site-packages


/Users/brandendunbar/Desktop/test/venv/lib/python2.7/site-packages/IPython/nbconvert.py:13: ShimWarning: The `IPython.nbconvert` package has been deprecated. You should import from nbconvert instead.
  "You should import from nbconvert instead.", ShimWarning)
/Users/brandendunbar/Desktop/test/venv/lib/python2.7/site-packages/publisher/publisher.py:53: UserWarning: Did you "Save" this notebook before running this command? Remember to save, always save.
  warnings.warn('Did you "Save" this notebook before running this command? '
/Users/brandendunbar/Desktop/test/venv/lib/python2.7/site-packages/publisher/publisher.py:58: UserWarning: Your URL has more than 2 parts... are you sure?
  warnings.warn('Your URL has more than 2 parts... are you sure?')
